In [1]:
import psycopg2
import sys
from sqlalchemy import create_engine
import pandas as pd
from setting import *


def init():
    if sys.platform == "win32":
        conn = psycopg2.connect(database=DB, user=USER, password=PASSWORD, host='localhost', port=PORT)
        cur = conn.cursor()
    else:
        print("The OS of this machine is " + sys.platform + ". Connecting to remote server...")
        # host = '143.248.109.99'
        conn = psycopg2.connect(database=DB, user=USER, password=PASSWORD, host=HOST, port=PORT)
        cur = conn.cursor()
    return conn, cur

engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(USER, PASSWORD, HOST,PORT,DB))
engine2 = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(USER, PASSWORD, HOST,PORT,DB2))
con, cur = init()

The OS of this machine is darwin. Connecting to remote server...


In [ ]:
locations = pd.read_sql_table('locations',con=engine2)
locations.to_sql('locations',if_exists='replace',con=engine, index=False)

In [ ]:
ws = pd.read_csv('./raw/WS_Munkwa.csv')

In [ ]:
ws.to_sql('WS_Munkwa',if_exists='replace',con=engine, index=False)

In [ ]:
lnis = pd.read_csv('./raw/lnis_family.csv')
lnis.to_sql('lnis_family',if_exists='replace',con=engine, index=False)

In [ ]:
lee_uuid = pd.read_csv('./preprocessing/munkwa_lee_uuid_dict.csv')

In [ ]:
len(set(lee_uuid['lee_uuid']))

In [ ]:
lee_uuid

In [ ]:
for idx in lee_uuid.index:
    luid = lee_uuid.iloc[idx]['lee_uuid']
    bid = lee_uuid.iloc[idx]['bid']
    cur.execute(
        '''UPDATE bangmok SET lee_uuid =(%s)
        WHERE bid = (%s)
        ''', (luid, bid))
con.commit()
cur.close()

In [ ]:
ws_uuid = pd.read_csv('./preprocessing/munkwa_ws_uuid_dict.csv')

In [ ]:
for idx in ws_uuid.index:
    id1= ws_uuid.iloc[idx]['id1']
    wuid = ws_uuid.iloc[idx]['ws_uuid']
    cur.execute(
        '''UPDATE ws_munkwa SET ws_uuid =(%s)
        WHERE id1 = (%s)
        ''', (wuid, str(id1)))
    con.commit()
cur.close()

In [ ]:
sc = pd.read_sql('select * from sillok_career inner join dates on dates.solardate=sillok_career.solardate', con=engine)

In [ ]:
scr = sc.groupby('spid')['rdate'].apply(list)

In [ ]:
scr

In [ ]:
merged_munkwa = pd.read_csv('./preprocessing/Merged_Munkwa.csv')

In [ ]:
merged_munkwa.to_sql('merged_munkwa', index=False, if_exists='append', con=engine)

In [ ]:
bc = pd.read_sql_table('bangmok_career', con=engine)

In [ ]:
a = sorted([int(i) for i in list(bc['bcid'].str.replace('bc','')) if i is not None])
for i, j in enumerate(a):
    if i+1 !=j:
        if i+2 !=j:
            print(i, j)

In [ ]:
sp = pd.read_excel('./preprocessing/figures.xlsx')

In [ ]:
sp.to_sql('sillok_person', con=engine, if_exists='append',index=False)

In [ ]:
sc = pd.read_sql_table('sillok_career', con=engine)

In [ ]:
records = pd.read_excel('./preprocessing/records.xlsx')

In [ ]:
set(records['관직']) - set({k:v for k, v in zip(sc['officelevel'], sc['gid'])}.keys())

In [ ]:
sc['scid'] = sc['scid'].str.replace('sc','').astype(int)

In [ ]:
len(set(records['관직_ID']) - set(sc['scid']))

In [ ]:
t.columns

In [ ]:
t = pd.merge(records, sc, left_on='관직_ID', right_on = 'scid', how='left')

In [ ]:
dates = pd.read_sql_table('dates',con=engine)
dates = {k:v for k, v in zip(dates['lunardate'], dates['solardate'])}
t['solardate'] = (t['서기년'].astype(str)+'-'+t['서기월'].astype(str).apply(lambda x: x.zfill(2))+'-'+t['서기일'].astype(str).apply(lambda x: x.zfill(2))+'L'+t['윤달여부'].astype(str)).map(dates)

In [ ]:
t[t['scid'] !=  t['관직_ID']]

In [ ]:
for i in t[t['기사ID'] !=  t['sid']].index:
    spid = t.iloc[i]['인물_ID']
    officelevel = t.iloc[i]['관직']
    typo = t.iloc[i]['관직현황']
    sid = t.iloc[i]['기사ID']
    scid = 'sc'+str(t.iloc[i]['관직_ID'])
    solardate = t.iloc[i]['solardate']
    gid = None
    cur.execute('''
    insert into sillok_career (spid, officelevel, type, sid, scid, solardate, gid)
    values ((%s), (%s), (%s), (%s), (%s), (%s), (%s))
    ''', (spid, officelevel, typo, sid, scid, solardate, gid))
con.commit()
con.close()

In [ ]:
for i in t[t['관직'] !=  t['officelevel']].index:
    spid = t.iloc[i]['인물_ID']
    officelevel = t.iloc[i]['관직']
    sid = t.iloc[i]['기사ID']
    scid = 'sc'+str(t.iloc[i]['관직_ID'])
    solardate = t.iloc[i]['solardate']
    cur.execute(
        '''UPDATE sillok_career SET spid =(%s),
        officelevel = (%s),
        sid = (%s),
        solardate = (%s)
        WHERE scid = (%s)
        ''', (spid, officelevel, sid, solardate, scid))
con.commit()
con.close()

In [ ]:
t

In [ ]:
for i in t[t['관직'] !=  t['officelevel']].index:
    officelevel = t.iloc[i]['관직']
    typo = t.iloc[i]['관직현황']
    sid = t.iloc['기사ID']
    

In [ ]:
sc = pd.read_sql_table('sillok_career',con=engine)
sp = pd.read_sql_table('sillok_person', con=engine)

In [ ]:
set(sc['spid']) - set(sp['spid'])

In [ ]:
sn = pd.read_sql_table('sillok_name', con=engine)

In [ ]:
set(sn['nameid']) - set(sp['spid'])